In [1]:
import numpy as np
from scipy.signal import lfilter
from scipy.signal.windows import hann
from scipy.linalg import toeplitz, solve_toeplitz
import os
import librosa
import tensorflow.keras.layers as tfl
import tensorflow as tf
from keras.callbacks import ReduceLROnPlateau

2024-07-01 12:25:09.734074: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-01 12:25:09.734157: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-01 12:25:09.882767: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
def lpc(x, order):
    R = np.correlate(x, x, mode='full')[len(x)-1:]
    R = R[:order+1]
    return solve_toeplitz((R[:order], R[:order]), -R[1:])

In [3]:
def gfmiaif(s_gvl, nv=16, ng=16, d=0.99, win=None):
    if win is None:
        win = hann(len(s_gvl))
    
    Lpf = nv + 1
    x_gvl = np.concatenate((np.linspace(-s_gvl[0], s_gvl[0], Lpf), s_gvl))
    idx_pf = slice(Lpf, len(x_gvl))
    
    al = [1, -d]
    s_gv = lfilter([1], al, s_gvl)
    x_gv = lfilter([1], al, x_gvl)
    
    ag1 = lpc(s_gv * win, 1)
    
    for _ in range(ng - 1):
        x_v1x = lfilter(ag1, [1], x_gv)
        s_v1x = x_v1x[idx_pf]
        ag1x = lpc(s_v1x * win, 1)
        ag1 = np.convolve(ag1, ag1x)
    
    x_v1 = lfilter(ag1, [1], x_gv)
    s_v1 = x_v1[idx_pf]
    av1 = lpc(s_v1 * win, nv)
    
    x_g1 = lfilter(av1, [1], x_gv)
    s_g1 = x_g1[idx_pf]
    ag = lpc(s_g1 * win, ng)
    
    x_v = lfilter(ag, [1], x_gv)
    s_v = x_v[idx_pf]
    av = lpc(s_v * win, nv)
    
    return av, ag, al

In [5]:
def generate_train_test_instances(data_folder,num_speakers=5,num_utterances=20,sr=11000,hop_length=512,frame_length=2048,verbose=True):
    x=[[],[]]
    y_ret=[[],[]]
    
    total_speaker = 0
    for speaker in os.listdir(data_folder):
        total_utterances = 0
        for vidID in os.listdir(os.path.join(data_folder, speaker)):
            for file in os.listdir(os.path.join(data_folder, speaker, vidID)):
                if file.endswith(".wav"):
                    wav_file_path = os.path.join(data_folder, speaker, vidID, file)
                    
                    y, sr = librosa.load(wav_file_path, sr=sr)
#                     frames = librosa.util.frame(y, frame_length=frame_length, hop_length=hop_length).T
                    vt_coeff,glot_coeff,_=gfmiaif(y)
#                     glot_coeff=[0 for _ in range(16)]
#                     for frame in frames:
#                         _,glot,_=gfmiaif(frame)
#                         cnt=0
#                         for i in glot:
#                             glot_coeff[cnt]+=i
#                             cnt+=1
                    
#                     cnt=0
#                     for i in glot_coeff:
#                         glot_coeff[cnt]=glot_coeff[cnt]/len(frames)
#                         cnt+=1
                        
                    x[0].append(vt_coeff)
                    x[1].append(glot_coeff)
                    y_ret[0].append(speaker)
                    y_ret[1].append(speaker)
                    
                    total_utterances+=1
                    
                    if total_utterances==num_utterances:
                        break
                        
            if total_utterances==num_utterances:
                        break
        
        total_speaker += 1
        if total_speaker != 0 and total_speaker % 1 == 0 and verbose:
            print(f"{total_speaker} speakers completed.\n")
        
        if total_speaker == num_speakers:
            break
    
    return x,y_ret

In [6]:
num_speakers=5
num_utterances=20
data_folder="/kaggle/input/voxceleb1train/wav"

In [7]:
x,y=generate_train_test_instances(data_folder,num_speakers,num_utterances)

1 speakers completed.

2 speakers completed.

3 speakers completed.

4 speakers completed.

5 speakers completed.



In [8]:
dict={}
i=0

unique=[]
for val in y[0]:
    if val not in unique:
        unique.append(val)

for val in unique:
    dict[val]=i
    i+=1

dict

{'id10116': 0, 'id11079': 1, 'id11123': 2, 'id10459': 3, 'id10484': 4}

In [9]:
from sklearn.model_selection import train_test_split

X_train_vt, X_test_vt, y_train_vt, y_test_vt = train_test_split(x[0],y[0] , 
                                   random_state=104,  
                                   test_size=0.2,  
                                   shuffle=True)

X_train_glot, X_test_glot, y_train_glot, y_test_glot = train_test_split(x[1],y[1] , 
                                   random_state=104,  
                                   test_size=0.2,  
                                   shuffle=True)

In [10]:
i=0
for _ in y_train_vt:
    y_train_vt[i]=dict[y_train_vt[i]]
    i=i+1

i=0
for _ in y_test_vt:
    y_test_vt[i]=dict[y_test_vt[i]]
    i=i+1
    
i=0
for _ in y_train_glot:
    y_train_glot[i]=dict[y_train_glot[i]]
    i=i+1

i=0
for _ in y_test_glot:
    y_test_glot[i]=dict[y_test_glot[i]]
    i=i+1

In [11]:
def dnn_model(input_shape,output_shape):
    input_data = tf.keras.Input(shape=input_shape)
    
    D1 = tfl.Dense(512, activation='relu')(input_data)
    DP1 = tfl.Dropout(0.5)(D1)
    
    D2 = tfl.Dense(256, activation='relu')(DP1)
    DP2 = tfl.Dropout(0.5)(D2)
    
    D3 = tfl.Dense(128, activation='relu')(DP2)
    DP3 = tfl.Dropout(0.5)(D3)
    
    outputs = tfl.Dense(output_shape, activation='softmax')(DP3)
    
    model = tf.keras.Model(inputs=input_data, outputs=outputs)
    return model

In [12]:
model_vt = dnn_model((len(X_train_vt[0]),),num_speakers)
model_vt.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [13]:
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=10, min_lr=0.00001)
history = model_vt.fit(np.array(X_train_vt),np.array(y_train_vt), batch_size=32, epochs=100, validation_data=(np.array(X_test_vt),np.array(y_test_vt)),callbacks=(reduce_lr))

Epoch 1/100
1/3 ━━━━━━━━━━━━━━━━━━━━ 12s 6s/step - accuracy: 0.2500 - loss: 1.6520

I0000 00:00:1719839385.060397     128 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1719839385.076550     128 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.2229 - loss: 1.6720 

W0000 00:00:1719839390.434220     131 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


3/3 ━━━━━━━━━━━━━━━━━━━━ 12s 3s/step - accuracy: 0.2172 - loss: 1.6751 - val_accuracy: 0.1000 - val_loss: 1.6201 - learning_rate: 0.0010
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.1945 - loss: 1.6709 - val_accuracy: 0.1000 - val_loss: 1.6195 - learning_rate: 0.0010
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.2375 - loss: 1.5988 - val_accuracy: 0.1000 - val_loss: 1.6274 - learning_rate: 0.0010
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.2453 - loss: 1.6341 - val_accuracy: 0.1000 - val_loss: 1.6284 - learning_rate: 0.0010
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.3047 - loss: 1.5733 - val_accuracy: 0.2000 - val_loss: 1.6208 - learning_rate: 0.0010
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.2484 - loss: 1.5378 - val_accuracy: 0.2500 - val_loss: 1.6082 - learning_rate: 0.0010
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.1586 - loss: 1.6214 - val_accuracy: 0.2000 - val_

In [14]:
model_glot = dnn_model((len((X_train_glot[0]),),),num_speakers)
model_glot.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [15]:
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=10, min_lr=0.00001)
history = model_glot.fit(np.array(X_train_glot),np.array(y_train_glot), batch_size=32, epochs=100, validation_data=(np.array(X_test_glot),np.array(y_test_glot)),callbacks=(reduce_lr))

Epoch 1/100
1/3 ━━━━━━━━━━━━━━━━━━━━ 9s 5s/step - accuracy: 0.1562 - loss: 1.6434

W0000 00:00:1719839427.604907     130 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.1667 - loss: 1.6425

W0000 00:00:1719839432.365185     131 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - accuracy: 0.1719 - loss: 1.6409 - val_accuracy: 0.1500 - val_loss: 1.6260 - learning_rate: 0.0010
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.2469 - loss: 1.6288 - val_accuracy: 0.1000 - val_loss: 1.6249 - learning_rate: 0.0010
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.2672 - loss: 1.5600 - val_accuracy: 0.1000 - val_loss: 1.6260 - learning_rate: 0.0010
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.2164 - loss: 1.5938 - val_accuracy: 0.1000 - val_loss: 1.6261 - learning_rate: 0.0010
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.2070 - loss: 1.6119 - val_accuracy: 0.1000 - val_loss: 1.6249 - learning_rate: 0.0010
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.2398 - loss: 1.5990 - val_accuracy: 0.1000 - val_loss: 1.6257 - learning_rate: 0.0010
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.1977 - loss: 1.5898 - val_accuracy: 0.1000 - val_

In [16]:
def create_train_triplets(X_train, y_train, num_triplets):
    triplets = []
    labels = np.unique(y_train)
    
    for _ in range(num_triplets):
        anchor_label = np.random.choice(labels)
        negative_label = np.random.choice(labels[labels != anchor_label])
        
        anchor_indices = np.where(y_train == anchor_label)[0]
        positive_indices = np.where(y_train == anchor_label)[0]
        negative_indices = np.where(y_train == negative_label)[0]
        
        anchor = X_train[np.random.choice(anchor_indices)]
        positive = X_train[np.random.choice(positive_indices)]
        negative = X_train[np.random.choice(negative_indices)]
        
        triplets.append((anchor, positive, negative))
    
    return np.array(triplets)

In [17]:
def create_test_triplets(X_test, y_test, num_triplets):
    triplets = []
    range_test = len(y_test)
    idx = np.arange(range_test)
    
    for _ in range(num_triplets):
        label_1 = np.random.choice(idx)
        label_2 = np.random.choice(idx)
        
        anchor = X_test[label_1]
        inp = X_test[label_2]
        out = 1 if y_test[label_1] == y_test[label_2] else 0
        
        triplets.append((anchor, inp, out))
        
    return np.array(triplets, dtype=object) 

In [19]:
def dnn_model_triplet_loss(input_shape):
    input_data = tf.keras.Input(shape=input_shape)
    
    D1 = tfl.Dense(512, activation='relu')(input_data)
    DP1 = tfl.Dropout(0.5)(D1)
    
    D2 = tfl.Dense(256, activation='relu')(DP1)
    DP2 = tfl.Dropout(0.5)(D2)
    
    outputs = tfl.Dense(128, activation='relu')(DP2)
        
    model = tf.keras.Model(inputs=input_data, outputs=outputs)
    return model

input_shape = (16,)
base_model = dnn_model_triplet_loss(input_shape)

In [20]:
def triplet_loss(y_true, y_pred, margin=0.2):
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis=-1)
    neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis=-1)
    loss = tf.maximum(pos_dist - neg_dist + margin, 0.0)
    return tf.reduce_mean(loss)

In [21]:
anchor_input = tfl.Input(shape=input_shape, name='anchor_input')
positive_input = tfl.Input(shape=input_shape, name='positive_input')
negative_input = tfl.Input(shape=input_shape, name='negative_input')

encoded_anchor = base_model(anchor_input)
encoded_positive = base_model(positive_input)
encoded_negative = base_model(negative_input)

triplet_model_vt = tf.keras.Model(inputs=[anchor_input, positive_input, negative_input],
                      outputs=[encoded_anchor, encoded_positive, encoded_negative])

triplet_model_vt.compile(optimizer='adam', loss=triplet_loss)

triplet_model_glot = tf.keras.Model(inputs=[anchor_input, positive_input, negative_input],
                      outputs=[encoded_anchor, encoded_positive, encoded_negative])

triplet_model_glot.compile(optimizer='adam', loss=triplet_loss)

In [22]:
train_num=500
test_num=100

In [24]:
triplets_train=create_train_triplets(X_train_vt,y_train_vt,train_num)

anchor_images = triplets_train[:, 0]
positive_images = triplets_train[:, 1]
negative_images = triplets_train[:, 2]

triplet_model_vt.fit([anchor_images, positive_images, negative_images], 
                  np.zeros((train_num, )),
                  batch_size=32, 
                  epochs=100)

Epoch 1/100
 1/16 ━━━━━━━━━━━━━━━━━━━━ 58s 4s/step - loss: 0.0000e+00

W0000 00:00:1719840173.978069     128 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 234ms/step - loss: 0.7521 
Epoch 2/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4252 
Epoch 3/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1631     
Epoch 4/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1308 
Epoch 5/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1873     
Epoch 6/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7530 
Epoch 7/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8009 
Epoch 8/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6571     
Epoch 9/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5076 
Epoch 10/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3473 
Epoch 11/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0957 
Epoch 12/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3561 
Epoch 13/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8023 
Epoch 14/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3156 
Epoch 15/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - l

In [26]:
triplets_test=create_test_triplets(X_test_vt,y_test_vt,test_num)

true_pos = 0
false_pos = 0

anchor_input_test = np.stack(triplets_test[:, 0])
positive_input_test = np.stack(triplets_test[:, 1])
labels_test = triplets_test[:, 2]

encoded_anchor = base_model.predict(anchor_input_test)
encoded_positive = base_model.predict(positive_input_test)

distances = np.linalg.norm(encoded_anchor - encoded_positive, axis=1)
threshold = 0.1

for i in range(test_num):
    print(str(distances[i])+" : "+str(labels_test[i]))
    if distances[i] < threshold and labels_test[i] == 1:
        true_pos += 1
    if distances[i] >= threshold and labels_test[i] == 0:
        false_pos += 1

total_pos=np.sum(labels_test)

accuracy = (true_pos+false_pos) / test_num
true_pos = (true_pos) / total_pos
false_pos = (false_pos) / (test_num-total_pos)
print("True Positives: {:.2f}".format(true_pos))
print("False Positives: {:.2f}".format(false_pos))
print("Accuracy: {:.2f}".format(accuracy))

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
0.14346313 : 0
0.099603996 : 0
0.105038576 : 0
0.09900102 : 0
0.053166863 : 0
0.11788099 : 0
0.092042446 : 0
0.073330656 : 0
0.11864111 : 0
0.10012665 : 0
0.17927936 : 0
0.18419698 : 0
0.105038576 : 0
0.13870403 : 0
0.022056013 : 0
0.14346313 : 0
0.0 : 1
0.05461231 : 0
0.06586469 : 1
0.06078943 : 1
0.21777856 : 0
0.20507592 : 0
0.1682442 : 1
0.28834218 : 1
0.0 : 1
0.16518576 : 0
0.1682442 : 1
0.1699624 : 0
0.158834 : 1
0.2670228 : 0
0.13657051 : 0
0.17158702 : 0
0.074541025 : 0
0.110594116 : 0
0.25225002 : 0
0.25225002 : 0
0.07505315 : 0
0.13687868 : 0
0.26595247 : 1
0.08769653 : 0
0.08581713 : 0
0.112412125 : 0
0.08221407 : 0
0.27476728 : 0
0.04488163 : 0
0.07877856 : 0
0.08161085 : 0
0.15476143 : 0
0.0 : 1
0.17392409 : 1
0.22557236 : 0
0.08432877 : 0
0.18225196 : 1
0.30604276 : 1
0.17158702 : 0
0.20553087 : 0
0.12723847 : 0
0.044654004 : 0
0.14346313 : 0
0.076332465 : 1
0.08251526 : 0
0.16295101 : 0
0.0309303

In [27]:
triplets_train=create_train_triplets(X_train_glot,y_train_glot,train_num)

anchor_images = triplets_train[:, 0]
positive_images = triplets_train[:, 1]
negative_images = triplets_train[:, 2]

triplet_model_glot.fit([anchor_images, positive_images, negative_images], 
                  np.zeros((train_num, )),
                  batch_size=32, 
                  epochs=100)

Epoch 1/100
 1/16 ━━━━━━━━━━━━━━━━━━━━ 55s 4s/step - loss: 0.1140

W0000 00:00:1719840396.964490     131 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 214ms/step - loss: 0.1932
Epoch 2/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2747 
Epoch 3/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1687 
Epoch 4/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1929 
Epoch 5/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2009     
Epoch 6/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1676 
Epoch 7/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2148 
Epoch 8/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1873 
Epoch 9/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1482 
Epoch 10/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2522 
Epoch 11/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1505     
Epoch 12/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1918 
Epoch 13/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1332 
Epoch 14/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3783 
Epoch 15/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 

In [29]:
triplets_test=create_test_triplets(X_test_glot,y_test_glot,test_num)

true_pos = 0
false_pos = 0

anchor_input_test = np.stack(triplets_test[:, 0])
positive_input_test = np.stack(triplets_test[:, 1])
labels_test = triplets_test[:, 2]

encoded_anchor = base_model.predict(anchor_input_test)
encoded_positive = base_model.predict(positive_input_test)

distances = np.linalg.norm(encoded_anchor - encoded_positive, axis=1)
threshold = 0.04

for i in range(test_num):
    print(str(distances[i])+" : "+str(labels_test[i]))
    if distances[i] < threshold and labels_test[i] == 1:
        true_pos += 1
    if distances[i] >= threshold and labels_test[i] == 0:
        false_pos += 1

total_pos=np.sum(labels_test)

accuracy = (true_pos+false_pos) / test_num
true_pos = (true_pos) / total_pos
false_pos = (false_pos) / (test_num-total_pos)
print("True Positives: {:.2f}".format(true_pos))
print("False Positives: {:.2f}".format(false_pos))
print("Accuracy: {:.2f}".format(accuracy))

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
0.0 : 1
0.14351217 : 0
0.043729994 : 0
0.03356261 : 1
0.048830442 : 0
0.04833459 : 0
0.0 : 1
0.024110246 : 1
0.08455219 : 0
0.041540228 : 0
0.05214638 : 0
0.12913454 : 0
0.042492796 : 0
0.028051985 : 1
0.07779462 : 0
0.0612921 : 0
0.1683823 : 0
0.0641909 : 1
0.034769658 : 1
0.03356261 : 1
0.04157321 : 0
0.123995945 : 0
0.044330794 : 0
0.037613865 : 0
0.06900022 : 0
0.031190963 : 1
0.12620609 : 0
0.22667472 : 1
0.12142745 : 0
0.14583592 : 0
0.054088358 : 0
0.055003952 : 0
0.055003952 : 0
0.06418631 : 0
0.12805204 : 1
0.0 : 1
0.03880233 : 0
0.0612921 : 0
0.0 : 1
0.1683823 : 0
0.048968725 : 0
0.056577828 : 1
0.041540228 : 0
0.02729711 : 0
0.045659352 : 0
0.038979236 : 1
0.044330794 : 0
0.02307883 : 1
0.16368064 : 0
0.019841665 : 1
0.04833459 : 0
0.14583592 : 0
0.054088358 : 0
0.123995945 : 0
0.040786847 : 0
0.04510211 : 0
0.03356261 : 1
0.0807776 : 0
0.057699762 : 0
0.05214638 : 0
0.09744793 : 0
0.01960194 : 0
0.0